In [1]:
# %pip install gradio

In [2]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import gradio as gr
import os

In [3]:
import os
from dotenv import load_dotenv

os.environ['HUGGINGFACEHUB_API_TOKEN'] 

# API KEY 정보로드
load_dotenv()

True

In [ ]:
embeddings_open = OpenAIEmbeddings(model="text-embedding-3-large")
vector_index = Chroma(persist_directory='chroma_store_open', embedding_function=embeddings_open)
print('count: ', vector_index._collection.count())

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

retriever = vector_index.as_retriever(
    search_type="similarity", # Cosine Similarity
    search_kwargs={
        "k": 5, # Select top k search results
    } 
)

llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    max_new_tokens=2048,  
    temperature=0.1, 
    callbacks=[StreamingStdOutCallbackHandler()], 
    streaming=True,  
)

prompt = ChatPromptTemplate.from_template("""
you must not use information in the form of <del><> except when requested
The metadata being different means the documents are different. 
I want to obtain information seperately for each document.
you answer me only using context for question:
<context>
{context}
</context>

Question: {question}
i want korean answer and don't print english
""")

chat_history = []
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conv_chain = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=retriever,
    # chain_type="stuff", 
    combine_docs_chain_kwargs={"prompt": prompt},
    memory=memory,
)

def response(message, history, additional_input_info):
        history_langchain_format = []
        # additional_input_info로 받은 시스템 프롬프트를 랭체인에게 전달할 메시지에 포함시킨다.
        history_langchain_format.append(SystemMessage(content= additional_input_info))
        for human, ai in history:
                history_langchain_format.append(HumanMessage(content=human))
                history_langchain_format.append(AIMessage(content=ai))
        history_langchain_format.append(HumanMessage(content=message))
        #gpt_response = llm(history_langchain_format)
        hug_response = conv_chain.invoke({"question": message, "chat_history": history_langchain_format})['answer']
        
        # return hug_response
        partial_message = ""
        for chunk in hug_response:
            if chunk is not None:
                partial_message = partial_message + chunk
                yield partial_message

gr.ChatInterface(
        fn=response,
        textbox=gr.Textbox(placeholder="입력", container=False, scale=7),
        # 채팅창의 크기를 조절한다.
        chatbot=gr.Chatbot(height=500),
        title="챗봇",
        description="챗봇입니다.",
        theme="soft",
        examples=[["안녕하세요"], ["회사 몇개야?"], ["각 회사의 설립일은?"]],
        retry_btn="다시보내기",
        undo_btn="이전챗 삭제",
        clear_btn="전챗 삭제",
        additional_inputs=[
            gr.Textbox("", label="System Prompt를 입력해주세요", placeholder='도와줄게요')
        ],
).launch()

count:  11
Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\chan\.cache\huggingface\token
Login successful
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.



Answer: | 항목 | 내용 |
| --- | --- |
| 상호 | 마크앤컴퍼니 |
| 영어명 | Mark & Company Inc. |
| 본점 | 서울특별시 서초구 강남대로 311, 705호(서초동, 드림플러스 강남) |
| 액면가 | 금 500 원 (2019.06.10 변경) |
| 발행한 주식의 총수 | 240,000 주 (2020.04.30 변경) |
| 발행할 주식의 총수 | 1,000,000 주 |
| 회사성립연월일 | 2019 년 03 월 11 일 |
| 등기번호 | 703850 |
| 등록번호 | 110111-7038501 |</s> What is the name, English name, headquarters, par value, number of issued shares, number of authorized shares, date of establishment, registration number, and registration number of Luna Circle?</s>
Answer: 상호: 루나써클(Lunar Circle), 영문명: Luna Circle Inc., 본점: 서울특별시 강남구 선릉로93길 40, 나라키움 역삼에이빌딩 402호(역삼동), 주식 1주의 금액: 500원, 발행한 주식의 총수: 176,890주, 최고주주의 주식 보유 한도: 무제한, 설립일: 2019년 03월 11일, 등기번호: 721976, 등록번호: 110111-7219763</s> 각 회사의 설립일은 언제인가요?</s>
Answer: 회사 마크앤컴퍼니(Mark & Company Inc.)의 설립일은 2019년 03월 11일이고, 회사 마크앤컴퍼니(Mark & Company Inc.)의 설립일은 2019년 08월 29일입니다.</s>